<a href="https://colab.research.google.com/github/AnandInguva/beam/blob/inference-notebook/examples/notebooks/documentation/transforms/python/elementwise/run_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License")
# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements. See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership. The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied. See the License for the
# specific language governing permissions and limitations
# under the License.

# RunInference


In this notebook, we walk through the use of the RunInference transform.
The transform and its accompanying [ModelHandler](https://beam.apache.org/releases/pydoc/current/apache_beam.ml.inference.base.html#apache_beam.ml.inference.base.ModelHandler) classes handle the following tasks:


*   Optimizing loading models from popular frameworks.
*   Batching examples in a scalable fashion.


This notebook illustrates:
*   Use Pytorch RunInference transform on un-keyed data.
*   Use Pytorch RunInference transform on keyed data.
*   Use Sklearn RunInference transform on un-keyed data.
*   Use RunInference transform on keyed data.

The linear regression model used in this notebook is trained on data that correspondes to the 5 times table; that is,`y = 5x`.

### Dependencies

The RunInference library is available in Apache Beam version <b>2.40</b> or later.

Pytorch module is needed to use Pytorch RunInference API. use `pip` to install Pytorch.

In [ ]:
%pip install apache_beam>=2.40.0 torch --quiet

In [ ]:
import csv
import os
import pickle
import torch
from typing import Tuple

import apache_beam as beam
import numpy
import numpy as np
from apache_beam.ml.inference.base import KeyedModelHandler
from apache_beam.ml.inference.base import PredictionResult
from apache_beam.ml.inference.base import RunInference
from apache_beam.ml.inference.pytorch_inference import PytorchModelHandlerTensor
from apache_beam.ml.inference.pytorch_inference import PytorchModelHandlerKeyedTensor
from apache_beam.ml.inference.sklearn_inference import ModelFileType
from apache_beam.ml.inference.sklearn_inference import SklearnModelHandlerNumpy
from apache_beam.options.pipeline_options import PipelineOptions
from sklearn import linear_model

## Create data and Pytorch models for RunInference transform

In [ ]:
save_model_dir_multiply_five = 'five_times_table_torch.pt'

# Linear regression model in Pytorch.
class LinearRegression(torch.nn.Module):
    def __init__(self, input_dim=1, output_dim=1):
        super().__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)  
    def forward(self, x):
        out = self.linear(x)
        return out

# train data for 5 times model
x = numpy.arange(0, 100, dtype=numpy.float32).reshape(-1, 1)
y = (x * 5).reshape(-1, 1)

five_times_model = LinearRegression()
optimizer = torch.optim.Adam(five_times_model.parameters())
loss_fn = torch.nn.L1Loss()

# Train the linear regression mode on 5 times data.
epochs = 10000
tensor_x = torch.from_numpy(x)
tensor_y = torch.from_numpy(y)
for epoch in range(epochs):
    y_pred = five_times_model(tensor_x)
    loss = loss_fn(y_pred, tensor_y)
    five_times_model.zero_grad()
    loss.backward()
    optimizer.step()

torch.save(five_times_model.state_dict(), save_model_dir_multiply_five)
print(os.path.exists(save_model_dir_multiply_five)) # verify if the model is saved

Data used for prediction on RunInference transforms.

In [ ]:
# unkeyed values are used for the prediction on RunInference API.
unkeyed_data = numpy.array([20, 40, 60, 90], dtype=numpy.float32).reshape(-1, 1)
# keyed values usef for the prediction on RunInference API
keyed_data = [("first_question", 105.00),
      ("second_question", 108.00),
      ("third_question", 1000.00),
      ("fourth_question", 1013.00)]

# Pytorch RunInference API

Define Pytorch ModelHandler that handles `Unkeyed` data.

In [ ]:
torch_five_times_model_handler = PytorchModelHandlerTensor(
    state_dict_path=save_model_dir_multiply_five,
    model_class=LinearRegression,
    model_params={'input_dim': 1,
                  'output_dim': 1}
                  )
pipeline = beam.Pipeline()

with pipeline as p:
      (
      p 
      | "ReadInputData" >> beam.Create(unkeyed_data)
      | "ConvertNumpyToTensor" >> beam.Map(torch.Tensor)
      | "RunInferenceTorch" >> RunInference(torch_five_times_model_handler)
      | beam.Map(print)
      )

Define Pytorch ModelHandler that handles `Keyed` data.

In [ ]:
pipeline_options = PipelineOptions()
pipeline = beam.Pipeline(options=pipeline_options)

torch_five_times_model_handler = PytorchModelHandlerTensor(
    state_dict_path=save_model_dir_multiply_five,
    model_class=LinearRegression,
    model_params={'input_dim': 1,
                  'output_dim': 1}
                  )
keyed_torch_five_times_model_handler = KeyedModelHandler(torch_five_times_model_handler)

with pipeline as p:
  (p
    | "ReadInputData" >> beam.Create(keyed_data)
    | "ConvertIntToTensor" >> beam.Map(lambda x: (x[0], torch.Tensor([x[1]])))
    | "RunInferenceTorch" >> RunInference(keyed_torch_five_times_model_handler)
    | beam.Map(print)
   )

## Create the data and the Sklearn model.


In [ ]:
# Input data to train the sklearn model.
x = numpy.arange(0, 100, dtype=numpy.float32).reshape(-1, 1)
y = (x * 5).reshape(-1, 1)

regression = linear_model.LinearRegression()
regression.fit(x,y)

sklearn_model_filename = 'sklearn_5x_model.pkl'
with open(sklearn_model_filename, 'wb') as f:
    pickle.dump(regression, f)

# Scikit-learn RunInference API.

Define Sklearn ModelHandler that handles `Unkeyed` data.

In [ ]:
sklearn_model_handler = SklearnModelHandlerNumpy(model_uri=sklearn_model_filename,
                                                 model_file_type=ModelFileType.PICKLE)

pipeline_options = PipelineOptions()
pipeline = beam.Pipeline(options=pipeline_options)

with pipeline as p:
  (
      p 
      | "ReadInputs" >> beam.Create(unkeyed_data)
      | "RunInferenceSklearn" >> RunInference(model_handler=sklearn_model_handler)
      | beam.Map(print)
  )

Define Sklearn ModelHandler that handles `Keyed` data.

In [ ]:
sklearn_model_handler = SklearnModelHandlerNumpy(model_uri=sklearn_model_filename,
                                                 model_file_type=ModelFileType.PICKLE)

keyed_sklearn_model_handler = KeyedModelHandler(sklearn_model_handler)

pipeline_options = PipelineOptions()
pipeline = beam.Pipeline(options=pipeline_options)

with pipeline as p:
  (
  p 
  | "ReadInputs" >> beam.Create(keyed_data)
  | "ExtractInputs" >> beam.Map(lambda x: (x[0], [x[1]]))
  | "RunInferenceSklearn" >> RunInference(model_handler=keyed_sklearn_model_handler)
  | beam.Map(print)
  )